Sqoop export (migración del HDFS hacia la BD)
===

* Última modificación: Mayo 19, 2022

Descarga de datos
---

In [1]:
file_name = "drivers.csv"
url = "https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/drivers/"
!wget --quiet {url + file_name} -P /tmp/

Creación de la base de datos
---

In [2]:
import sqlite3

conn = sqlite3.connect("sample.db")
cur = conn.cursor()

conn.executescript(
    """
    DROP TABLE IF EXISTS drivers;

    CREATE TABLE drivers (
        driverId       INT,
        name           STRING,
        ssn            STRING,
        location       STRING,
        certified      STRING,
        wage_plan      STRING
    );
    """
)
conn.commit()
conn.close()

In [3]:
!sqlite3 sample.db "SELECT name FROM sqlite_master WHERE type='table';"

drivers


Copia de los datos al HDFS
---

In [4]:
!tail +2 /tmp/drivers.csv > /tmp/drivers1.csv
!head /tmp/drivers1.csv

10,George Vetticaden,621011971,244-4532 Nulla Rd.,N,miles
11,Jamie Engesser,262112338,366-4125 Ac Street,N,miles
12,Paul Coddin,198041975,Ap #622-957 Risus. Street,Y,hours
13,Joe Niemiec,139907145,2071 Hendrerit. Ave,Y,hours
14,Adis Cesir,820812209,Ap #810-1228 In St.,Y,hours
15,Rohit Bakshi,239005227,648-5681 Dui- Rd.,Y,hours
16,Tom McCuch,363303105,P.O. Box 313- 962 Parturient Rd.,Y,hours
17,Eric Mizell,123808238,P.O. Box 579- 2191 Gravida. Street,Y,hours
18,Grant Liu,171010151,Ap #928-3159 Vestibulum Av.,Y,hours
19,Ajay Singh,160005158,592-9430 Nonummy Avenue,Y,hours


In [6]:
!hdfs dfs -copyFromLocal /tmp/drivers.csv /tmp/
!hdfs dfs -tail /tmp/drivers.csv

copyFromLocal: `/tmp/drivers.csv': File exists
Box 213- 8948 Nec Ave,Y,hours
27,Mark Lochbihler,392603159,8355 Ipsum St.,Y,hours
28,Olivier Renault,959908181,P.O. Box 243- 6509 Erat. Avenue,Y,hours
29,Teddy Choi,185502192,P.O. Box 106- 7003 Amet Rd.,Y,hours
30,Dan Rice,282307061,Ap #881-9267 Mollis Avenue,Y,hours
31,Rommel Garcia,858912101,P.O. Box 945- 6015 Sociis St.,Y,hours
32,Ryan Templeton,290304287,765-6599 Egestas. Av.,Y,hours
33,Sridhara Sabbella,967409015,Ap #477-2507 Sagittis Avenue,Y,hours
34,Frank Romano,391407216,Ap #753-6814 Quis Ave,Y,hours
35,Emil Siemes,971401151,321-2976 Felis Rd.,Y,hours
36,Andrew Grande,245303216,Ap #685-9598 Egestas Rd.,Y,hours
37,Wes Floyd,190504074,P.O. Box 269- 9611 Nulla Street,Y,hours
38,Scott Shaw,386411175,276 Lobortis Road,Y,hours
39,David Kaiser,967706052,9185 At Street,Y,hours
40,Nicolas Maillard,208510217,1027 Quis Rd.,Y,hours
41,Greg Phillips,308103116,P.O. Box 847- 5961 Arcu. Road,Y,hours
42,Randy Gelhausen,853302254,145-4200 In- Avenu

Exportación de una tabla
--

In [9]:
%%writefile /tmp/export.sh

sqoop export \
    --connect jdbc:sqlite:sample.db \
    --driver org.sqlite.JDBC \
    --table drivers \
    --export-dir /tmp/drivers.csv

Overwriting /tmp/export.sh


In [10]:
!bash /tmp/export.sh

Please set $HBASE_HOME to the root of your HBase installation.
Please set $HCAT_HOME to the root of your HCatalog installation.
Please set $ACCUMULO_HOME to the root of your Accumulo installation.
Please set $ZOOKEEPER_HOME to the root of your Zookeeper installation.
Note: /tmp/sqoop-root/compile/f95caf311d0e135863a5009cee7c9368/drivers.java uses or overrides a deprecated API.
Note: Recompile with -Xlint:deprecation for details.
22/05/19 21:07:42 ERROR mapreduce.ExportJobBase: Export job failed!
22/05/19 21:07:42 ERROR tool.ExportTool: Error during export: 
Export job failed!
	at org.apache.sqoop.mapreduce.ExportJobBase.runExport(ExportJobBase.java:445)
	at org.apache.sqoop.manager.SqlManager.exportTable(SqlManager.java:931)
	at org.apache.sqoop.tool.ExportTool.exportTable(ExportTool.java:80)
	at org.apache.sqoop.tool.ExportTool.run(ExportTool.java:99)
	at org.apache.sqoop.Sqoop.run(Sqoop.java:147)
	at org.apache.hadoop.util.ToolRunner.run(ToolRunner.java:76)
	at org.apache.sqoop.Sqoop

In [11]:
!sqlite3 sample.db "select * from drivers" | head

In [ ]:
!rm sample.db